Below is the test for evaluating the Boys Function.


In [6]:
using SpecialFunctions


function Boys(m::Int64,T::Float64)::Float64 # Boys function, used in the calculation of the electron repulsion integral
    return T^(-(m+0.5)) * (gamma(m+0.5) - gamma(m+0.5, T))/2  
end



# Helper function to sum over an interval with a given step size, used in the Romberg integration
function funcSum(func::Function, start::Float64, step::Float64, stop::Float64)::Float64
    s = 0.0
    x = start
    while x <= stop
        s += func(x)
        x += step
    end
    return s
end


# Romberg integration, the function is used to calculate Boys function, when the distance is small, we need to use numerical integration instead of Gamma function.
function romberg(a::Float64, b::Float64, M::Int, func::Function, varepsilon::Float64)::Float64
    h = b - a
    r1 = Vector{Float64}(undef, M)
    r2 = similar(r1)

    # Initial estimate using trapezoidal rule
    r1[1] = (func(a) + func(b)) * h / 2

    for k in 1:M-1
        h_k = h / 2^k
        # Estimate the sum of the function evaluated at new points
        sum_a = sum(func, a + h_k/2:h_k:b-h_k)
        
        # Calculate the next level of Romberg integration
        r2[1] = 0.5 * (r1[1] + sum_a * h_k)

        for j in 1:k
            r2[j+1] = r2[j] + (r2[j] - r1[j]) / (4^j - 1)
        end

        if abs(r2[k+1] - r1[k]) < varepsilon
            return r2[k+1]
        end
        
        # Copy r2 to r1 for the next iteration
        copyto!(r1, r2)
        if k == M-1
            println("Romberg did not converge")
        end
    end

    return r2[end]
end


function boys_diff(m::Int64, T::Float64,x::Float64)::Float64
    return x^(2*m) * exp(-T*x^2)
end

fix_boys_diff(x::Float64) = boys_diff(2, 0.1, x)

println(romberg(0.0, 1.0, 21, fix_boys_diff, 1e-6)," ",Boys(2, 0.0001))

Romberg did not converge
0.18624994948972604 0.19998558364875405
